In [1]:
import os
os.environ["KAGGLE_CONFIG_DIR"] = "/Users/atintaya/MyProjects/data_analysis_bikes"

In [2]:
import duckdb
import zipfile
from plotly import express as px

In [3]:
# Downloading dataset from kaggle
!kaggle datasets download -d hmavrodiev/london-bike-sharing-dataset

london-bike-sharing-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
zfile_name = 'london-bike-sharing-dataset.zip'

with zipfile.ZipFile(zfile_name, 'r') as file:
    file.extractall()

In [5]:
conn = duckdb.connect()

In [6]:
df_bikes = conn.execute('FROM london_merged.csv').df()

In [ ]:
df_bikes.head(2)

In [ ]:
df_bikes.info()

In [ ]:
df_bikes.describe()

In [ ]:
fig = px.line(df_bikes, x='timestamp',y='cnt')
fig.show()

In [ ]:
'''
Scatter Plot: (2, CATEGORIA) Relationship between two variables. I.e. Car's speed and fuel efficiency
Line Plot: (2, MEDIDOR y TIEMPO) Show trends or changes over time. I.e. Stock market trends over year.
Bar Chart: (2, MEDIDOR y CATEGORIA) Comparing Categorical data or showing discrete values. I.e. Sales for different products
Histogram: (VARIABLE CONTINUA) Distribution of CONTINUOUS varieble. I.e. Age distribution in poppulation
Box Plot (Distribución por CATEGORIA): Distribution of CONTINUOUS varieble throught quantiles. I.e. House prices on different neirghboors
Heat Map: CORRELATIONs between different variables. I.e. Correlations between different stocks
Area Plot: Changes in several variables over time. I.e. Temperature Changes
Bubble Chart: Relationships between 3 variables using marker size. Country GDP(x), Poppulation(y) and Area(size)
'''
df_bikes.head(2)
# fig = px.line(df_bikes, x='timestamp',y='cnt')
# fig.show()

In [ ]:
df_bikes.shape

In [ ]:
df_bikes.weather_code.value_counts()

In [ ]:
df_bikes.season.value_counts()

In [ ]:
new_columns = {
    'timestamp': 'time',
    'cnt': 'count',
    't1': 'temp_real',
    't2': 'temp_feels',
    'hum': 'humidity_percent',
    'wind_speed': 'wind_speed_kmh',
    'weather_code': 'weather',
}
df_bikes.rename(columns=new_columns, inplace=True)
df_bikes.head(2)

In [ ]:
df_bikes.humidity_percent = df_bikes.humidity_percent/100

In [ ]:
season_dict = {
    '0.0': 'spring',
    '1.0': 'summer',
    '2.0': 'autum',
    '3.0': 'winter',
}

weather_dict = {
    '1.0':'Clear',
    '2.0':'Scatered Clouds',
    '3.0':'Broken Clouds',
    '4.0':'Cloudy',
    '7.0':'Rain',
    '10.0':'Rain with thunderstorm',
    '26.0':'Snowfall',
}

df_bikes.season = df_bikes.season.astype('str')
df_bikes.season = df_bikes.season.map(season_dict)
df_bikes.weather = df_bikes.weather.astype('str')
df_bikes.weather = df_bikes.weather.map(weather_dict)

df_bikes.head(2)

In [ ]:
df_bikes.head()

In [ ]:
df_bikes.columns

In [90]:
query = """
SELECT
    timestamp as time,
    date_trunc('day', timestamp) as daytime,
    YEAR(timestamp) as YEAR,
    HOUR(timestamp) as hours_contracted,
    strftime(timestamp, '%A') as days_contracted,
    YEAR(timestamp) || LPAD(MONTH(timestamp),2,'0') AS monthly,
    cnt as count,
    t1 as temp_real,
    t2 as temp_feels,
    hum/100 as humidity_percent,
    wind_speed as wind_speed_kmh,
    CASE WHEN weather_code = '1.0' THEN 'Clear'
         WHEN weather_code = '2.0' THEN 'Scatered Clouds'
         WHEN weather_code = '3.0' THEN 'Broken Clouds'
         WHEN weather_code = '4.0' THEN 'Cloudy'
         WHEN weather_code = '7.0' THEN 'Rain'
         WHEN weather_code = '10.0' THEN 'Rain with thunderstorm'
         WHEN weather_code = '26.0' THEN 'Snowfall'
    END as weather,
    is_holiday,
    is_weekend,
    CASE WHEN season = '0.0' THEN 'spring'
         WHEN season = '1.0' THEN 'summer'
         WHEN season = '2.0' THEN 'autum'
         WHEN season = '3.0' THEN 'winter'
    END as season
FROM df_bikes
"""
df_bikes_transformed = conn.execute(query).df()
df_bikes_transformed.head(2)

,time,daytime,YEAR,hours_contracted,days_contracted,monthly,count,temp_real,temp_feels,humidity_percent,wind_speed_kmh,weather,is_holiday,is_weekend,season
0,2015-01-04 00:00:00,2015-01-04,2015,0,Sunday,201501,182,3.0,2.0,0.93,6.0,Broken Clouds,0.0,1.0,winter
1,2015-01-04 01:00:00,2015-01-04,2015,1,Sunday,201501,138,3.0,2.5,0.93,5.0,Clear,0.0,1.0,winter


In [91]:
df_bikes_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17414 entries, 0 to 17413
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   time              17414 non-null  datetime64[us]
 1   daytime           17414 non-null  datetime64[us]
 2   YEAR              17414 non-null  int64         
 3   hours_contracted  17414 non-null  int64         
 4   days_contracted   17414 non-null  object        
 5   monthly           17414 non-null  object        
 6   count             17414 non-null  int64         
 7   temp_real         17414 non-null  float64       
 8   temp_feels        17414 non-null  float64       
 9   humidity_percent  17414 non-null  float64       
 10  wind_speed_kmh    17414 non-null  float64       
 11  weather           17414 non-null  object        
 12  is_holiday        17414 non-null  float64       
 13  is_weekend        17414 non-null  float64       
 14  season            1741

In [ ]:
df_bikes_transformed.weather.value_counts()

# PLOTLY PLOTS:
- PLOTLY VIOLIN PLOT
- PLOTLY BOXPLOT
- PLOTLY HISTOGRAM
- PLOTLY SCATTER PLOT
- PLOTLY LINE PLOT
- PLOTLY BARPLOT

# PLOTLY VIOLIN PLOT

In [143]:
df_new = px.data.tips()
df_new.head(2)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3


In [145]:
fig = px.violin(df_new, y='total_bill')
fig.show()

In [147]:
fig = px.violin(df_new, y='total_bill', box=True,
                points='all')
fig.show()

In [150]:
fig = px.violin(df_new, y='tip', color='time',
                violinmode='overlay', hover_data=df_new.columns
                )
fig.show()

In [151]:
fig = px.strip(df_new, x='day', y='tip')
fig.show()

In [153]:
fig = px.violin(df_new, y='total_bill')
fig.show()

In [152]:
fig = px.violin(df_new, y='total_bill')
fig.update_traces(quartilemethod='exclusive')
fig.show()

In [154]:
fig = px.violin(df_new, y='total_bill')
fig.update_traces(quartilemethod='inclusive')
fig.show()

# PLOTLY BOXPLOT

In [133]:
query = 'SELECT YEAR, AVG(wind_speed_kmh) AS AVG_WIND_SPEED, weather, monthly, sum(count) AS total from df_bikes_transformed GROUP BY YEAR, weather, monthly ORDER BY MONTHLY'
df_bikes_monthly = conn.execute(query).df()
df_bikes_monthly.query("YEAR==2016", inplace=True)
px.box(df_bikes_monthly, y='total')

In [132]:
query = 'SELECT YEAR, AVG(wind_speed_kmh) AS AVG_WIND_SPEED, weather, monthly, sum(count) AS total from df_bikes_transformed GROUP BY YEAR, weather, monthly ORDER BY MONTHLY'
df_bikes_monthly = conn.execute(query).df()
df_bikes_monthly.query("YEAR==2016", inplace=True)
px.box(df_bikes_monthly, y='total',
       color='weather'
       )

In [130]:
query = 'SELECT YEAR, AVG(wind_speed_kmh) AS AVG_WIND_SPEED, weather, monthly, sum(count) AS total from df_bikes_transformed GROUP BY YEAR, weather, monthly ORDER BY MONTHLY'
df_bikes_monthly = conn.execute(query).df()
df_bikes_monthly.query("YEAR==2016", inplace=True)
px.box(df_bikes_monthly, x='weather', y='total',
       points='all'
       )

In [134]:
df_new = px.data.tips()
df_new.head(2)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3


In [138]:
fig = px.box(df_new, x='time', y='total_bill',
             points='outliers', color='smoker'
             )
fig.show()

In [140]:
fig = px.box(df_new, x='time', y='total_bill',
             points='outliers', color='smoker', notched=True,
             hover_data=['day'],
             title='Box Plot of Total Bill'
             )
fig.show()

- Getting only points from the boxplot

In [141]:
fig = px.strip(df_new, x='day', y='tip')
fig.show()

# PLOTLY HISTOGRAM

In [112]:
query = 'SELECT YEAR, AVG(wind_speed_kmh) AS AVG_WIND_SPEED, weather, monthly, sum(count) AS total from df_bikes_transformed GROUP BY YEAR, weather, monthly ORDER BY MONTHLY'
df_bikes_monthly = conn.execute(query).df()
df_bikes_monthly.query("YEAR==2016", inplace=True)
px.histogram(df_bikes_monthly, x='total', nbins=20, color='weather',
             histnorm='probability density',  # Parameters of distribution, not needed
             title='Total of Number of Bikes in a month')

In [110]:
query = 'SELECT YEAR, AVG(wind_speed_kmh) AS AVG_WIND_SPEED, weather, monthly, sum(count) AS total from df_bikes_transformed GROUP BY YEAR, weather, monthly ORDER BY MONTHLY'
df_bikes_monthly = conn.execute(query).df()
df_bikes_monthly.query("YEAR==2016", inplace=True)
px.histogram(df_bikes_monthly, x='monthly', y='total', color='weather')

In [82]:
query = 'SELECT YEAR, monthly, AVG(wind_speed_kmh) AS AVG_WIND_SPEED, days_contracted, sum(count) AS total from df_bikes_transformed GROUP BY YEAR, days_contracted, monthly ORDER BY MONTHLY'
df_bikes_monthly = conn.execute(query).df()
df_bikes_monthly.query("YEAR==2016", inplace=True)
px.histogram(df_bikes_monthly, x='days_contracted', y='total')

In [107]:
query = 'SELECT YEAR, daytime, season, weather, AVG(wind_speed_kmh) AS AVG_WIND_SPEED, sum(count) AS total from df_bikes_transformed GROUP BY YEAR, daytime, season, weather'
df_bikes_monthly = conn.execute(query).df()
df_bikes_monthly.query("YEAR==2016", inplace=True)
fig = px.histogram(df_bikes_monthly, x='daytime', y='total', color='season',
             title='Total of Number of Bikes per month in 2016')
fig.update_layout(bargap=0.1)
fig.show()

In [113]:
df_new = px.data.tips()
df_new.head(2)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3


In [115]:
px.histogram(df_new, x='total_bill', color='sex')

In [119]:
px.histogram(df_new, x='total_bill', y='tip', histfunc='avg',
             title='AVERAGE tip per total bill'
             )

In [123]:
fig = px.histogram(df_new, x='sex', y='total_bill', color='sex',
                   pattern_shape='smoker',
                   title='Total Bill per Sex, differenciated by smokers'
                   )
fig.show()

In [125]:
px.histogram(df_new, x='total_bill', y='tip', histfunc='avg',
             nbins=8, text_auto=True,
             title='AVERAGE tip per total bill'
             )

# PLOTLY SCATTER PLOT

In [45]:
df_bikes_transformed.columns

Index(['time', 'YEAR', 'hours_contracted', 'days_contracted', 'monthly',
       'count', 'temp_real', 'temp_feels', 'humidity_percent',
       'wind_speed_kmh', 'weather', 'is_holiday', 'is_weekend', 'season'],
      dtype='object')

In [72]:
query = 'SELECT YEAR, AVG(wind_speed_kmh) AS AVG_WIND_SPEED, weather, monthly, sum(count) AS total from df_bikes_transformed GROUP BY YEAR, weather, monthly ORDER BY MONTHLY'
df_bikes_monthly = conn.execute(query).df()
df_bikes_monthly.query("YEAR==2016", inplace=True)
px.scatter(df_bikes_monthly, x='AVG_WIND_SPEED', y='total', color='weather', symbol='weather',
        title='Coparison between Wind Speed and Total Rides, diferenciated by weather',
        marginal_x='histogram',
        marginal_y='rug'
        )

In [63]:
query = 'SELECT YEAR, AVG(wind_speed_kmh) AS AVG_WIND_SPEED, season, monthly, sum(count) AS total from df_bikes_transformed GROUP BY YEAR, season, monthly ORDER BY MONTHLY'
df_bikes_monthly = conn.execute(query).df()
df_bikes_monthly.query("YEAR==2016", inplace=True)
px.scatter(df_bikes_monthly, x='AVG_WIND_SPEED', y='season', color='total',
        title='Coparison between Wind Speed and Season, differenciated by Total Rides')

In [64]:
query = 'SELECT YEAR, AVG(wind_speed_kmh) AS AVG_WIND_SPEED, season, monthly, sum(count) AS total from df_bikes_transformed GROUP BY YEAR, season, monthly ORDER BY MONTHLY'
df_bikes_monthly = conn.execute(query).df()
df_bikes_monthly.query("YEAR==2016", inplace=True)
px.scatter(df_bikes_monthly, x='AVG_WIND_SPEED', y='season', size='total',
        title='Coparison between Wind Speed and Season, considering the size of Total Rides')

In [65]:
query = 'SELECT YEAR, AVG(wind_speed_kmh) AS AVG_WIND_SPEED, weather, season, monthly, sum(count) AS total from df_bikes_transformed GROUP BY YEAR, weather, season, monthly ORDER BY MONTHLY'
df_bikes_monthly = conn.execute(query).df()
df_bikes_monthly.query("YEAR==2016", inplace=True)
px.scatter(df_bikes_monthly, y='AVG_WIND_SPEED', x='season', color='weather', size='total',
        title='Coparison between Wind Speed and Season, colored by Weather and a size of Total Rides')

In [47]:
query = 'SELECT YEAR, AVG(temp_real) AS AVG_TEMP_REAL, AVG(temp_feels) AS AVG_TEMP_FEELS, monthly, sum(count) AS total from df_bikes_transformed GROUP BY YEAR, monthly ORDER BY MONTHLY'
df_bikes_monthly = conn.execute(query).df()
df_bikes_monthly.query("YEAR==2016", inplace=True)
px.scatter(df_bikes_monthly, x='AVG_TEMP_REAL', y='AVG_TEMP_FEELS',
        title='Coparison between Temperature Real and Feeled')

In [67]:
df_new = px.data.tips()
df_new.head(2)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3


In [71]:
fig = px.scatter(df_new, x='total_bill', y='tip',
                 color='smoker', facet_col='sex', facet_row='time')
fig.show()

In [74]:
fig = px.scatter(df_new, x='total_bill', y='tip',
                 trendline='ols') # Ordinary Least Squares
fig.show()

# PLOTLY LINE PLOT

In [22]:
df_bikes_transformed.columns

Index(['time', 'YEAR', 'hours_contracted', 'days_contracted', 'monthly',
       'count', 'temp_real', 'temp_feels', 'humidity_percent',
       'wind_speed_kmh', 'weather', 'is_holiday', 'is_weekend', 'season'],
      dtype='object')

In [37]:
query = 'SELECT YEAR, season, monthly, sum(count) AS total from df_bikes_transformed GROUP BY YEAR, season, monthly ORDER BY MONTHLY'
df_bikes_monthly = conn.execute(query).df()
df_bikes_monthly.query("YEAR==2016", inplace=True)
px.line(df_bikes_monthly, x='monthly', y='total', title='Total Bikes in 2016')

In [42]:
query = 'SELECT YEAR, weather, monthly, sum(count) AS total from df_bikes_transformed GROUP BY YEAR, weather, monthly ORDER BY MONTHLY'
df_bikes_monthly = conn.execute(query).df()
df_bikes_monthly.query("YEAR==2016", inplace=True)
fig = px.line(df_bikes_monthly, x='monthly', y='total', color='weather', markers=True,
        title='Total Bikes in 2016 by season')
fig.show()

In [43]:
df_bikes_transformed.columns

Index(['time', 'YEAR', 'hours_contracted', 'days_contracted', 'monthly',
       'count', 'temp_real', 'temp_feels', 'humidity_percent',
       'wind_speed_kmh', 'weather', 'is_holiday', 'is_weekend', 'season'],
      dtype='object')

In [44]:
query = 'SELECT YEAR, AVG(temp_real) AS AVG_TEMP_REAL, AVG(temp_feels) AS AVG_TEMP_FEELS, monthly, sum(count) AS total from df_bikes_transformed GROUP BY YEAR, monthly ORDER BY MONTHLY'
df_bikes_monthly = conn.execute(query).df()
df_bikes_monthly.query("YEAR==2016", inplace=True)
px.line(df_bikes_monthly, x='monthly', y=['AVG_TEMP_REAL', 'AVG_TEMP_FEELS'], markers=True,
        title='Total Bikes in 2016 by season')

In [39]:
df_bikes_monthly.query("monthly=='201608'")

,YEAR,weather,monthly,total
108,2016,Cloudy,201608,22514.0
109,2016,Rain,201608,41123.0
110,2016,Clear,201608,577541.0
111,2016,Broken Clouds,201608,111774.0
112,2016,Scatered Clouds,201608,384362.0


# PLOTLY BARPLOT

In [9]:
fig = px.bar(df_bikes_transformed, x='weather',color='weather', opacity=0.9,
             color_discrete_sequence=['red', 'blue', 'green', 'purple', 'black', 'brown', 'pink'])
fig.update_traces(marker_line_width = 0)
fig.show()

In [12]:
query = 'SELECT season, weather, sum(count) AS total from df_bikes_transformed group by season, weather'
df_bikes_season = conn.execute(query).df()
fig = px.bar(df_bikes_season, x='season', y='total', color='weather', text='total', title='Number of Bikes per Season',
             color_discrete_sequence=['red', 'blue', 'green', 'purple', 'black', 'brown', 'pink'])
fig.show()

In [21]:
wide_data = px.data.medals_wide()
px.bar(wide_data, x='nation', y=['gold', 'silver', 'bronze'], text='value',
       title = 'Wide Format Data')

In [ ]:
df_2016 = df_bikes_transformed.query("YEAR==2016")
df_2016.YEAR.value_counts()

fig = px.bar(df_2016, x='season', y='count', color='season', template='simple_white', hover_data=[], labels={'count': 'Total'})
fig.update_traces(marker_line_width = 0)
fig.show()

In [ ]:
query = """
SELECT
    YEAR::STRING AS YEAR,
    season,
    SUM(count) as total_bikes,
    avg(temp_real) avg_temp_real,
    avg(temp_feels) avg_temp_feels,
    avg(humidity_percent) avg_humidity_percent,
    avg(wind_speed_kmh) avg_wind_speed_kmh
FROM df_bikes_transformed
GROUP BY
    YEAR,
    season,
"""
df_bikes_season = conn.execute(query).df()
df_bikes_season.head(2)

In [ ]:
query = """
SELECT
    monthly,
    SUM(count) as total_bikes,
    avg(temp_real) avg_temp_real,
    avg(temp_feels) avg_temp_feels,
    avg(humidity_percent) avg_humidity_percent,
    avg(wind_speed_kmh) avg_wind_speed_kmh
FROM df_bikes_transformed
GROUP BY
    monthly
ORDER BY
    monthly
"""
df_bikes_monthly = conn.execute(query).df()
df_bikes_monthly.head(2)

# Area Plot

In [ ]:
fig = px.area(df_bikes_transformed, x='time', y='weather')
fig.show()

# Heatmap

In [ ]:
fig = px.density_heatmap(df_bikes_transformed, x='count', y='wind_speed_kmh')
fig.show()

# BOX PLOT

In [ ]:
df_bikes_transformed.columns

In [ ]:
fig = px.box(df_bikes_transformed, x='wind_speed_kmh')
fig.show()

In [ ]:
fig = px.box(df_bikes_transformed, x='weather', y='count')
fig.show()

In [ ]:
fig = px.box(df_bikes_transformed, x='season', y='count')
fig.show()

# HISTOGRAM

In [ ]:
fig = px.histogram(df_bikes_monthly, x='monthly', y='total_bikes')
fig.show()

# BAR PLOT

In [ ]:
fig = px.bar(df_bikes_season, x='season', y='total_bikes', color='YEAR')
fig.show()

In [ ]:
df_bikes_transformed.columns

In [ ]:
query = """
SELECT
    days_contracted,
    SUM(count) as total_bikes,
    avg(temp_real) avg_temp_real,
    avg(temp_feels) avg_temp_feels,
    avg(humidity_percent) avg_humidity_percent,
    avg(wind_speed_kmh) avg_wind_speed_kmh
FROM df_bikes_transformed
GROUP BY
    days_contracted
"""
df_bikes_days = conn.execute(query).df()
df_bikes_days.head(2)

In [ ]:
query = """
SELECT
    hours_contracted,
    SUM(count) as total_bikes,
    avg(temp_real) avg_temp_real,
    avg(temp_feels) avg_temp_feels,
    avg(humidity_percent) avg_humidity_percent,
    avg(wind_speed_kmh) avg_wind_speed_kmh
FROM df_bikes_transformed
GROUP BY
    hours_contracted
"""
df_bikes_hours = conn.execute(query).df()
df_bikes_hours.head(2)

# MULTIPLE PLOTS

In [ ]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Create subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=("Plot 1", "Plot 2"))

# Add traces to subplots
fig.add_trace(go.Scatter(x=df_bikes_days['days_contracted'], y=df_bikes_days['total_bikes'], mode='lines'), row=1, col=1)
fig.add_trace(go.Scatter(x=df_bikes_monthly['monthly'], y=df_bikes_monthly['total_bikes'], mode='lines'), row=1, col=2)

# Update layout
fig.update_layout(title='Multiple Plots Example', showlegend=False)

# Show the figure
fig.show()

# Line Plot

In [ ]:
fig = px.line(df_bikes_tp, x='days_contracted', y='total_bikes')
fig.show()

In [ ]:
query = """
SELECT
    monthly,
    weather,
    season,
    SUM(count) as total_bikes,
    avg(temp_real) avg_temp_real,
    avg(temp_feels) avg_temp_feels,
    avg(humidity_percent) avg_humidity_percent,
    avg(wind_speed_kmh) avg_wind_speed_kmh
FROM df_bikes_transformed
GROUP BY
    monthly,
    weather,
    season
"""
df_bikes_grouped = conn.execute(query).df()
df_bikes_grouped.head(2)

In [ ]:
'''
Scatter Plot: (2, CATEGORIA) Relationship between two variables. I.e. Car's speed and fuel efficiency
Line Plot: (2, MEDIDOR y TIEMPO) Show trends or changes over time. I.e. Stock market trends over year.
Bar Chart: (2, MEDIDOR y CATEGORIA) Comparing Categorical data or showing discrete values. I.e. Sales for different products
Histogram: (VARIABLE CONTINUA) Distribution of CONTINUOUS varieble. I.e. Age distribution in poppulation
Box Plot (Distribución por CATEGORIA): Distribution of CONTINUOUS varieble throught quantiles. I.e. House prices on different neirghboors
Heat Map: CORRELATIONs between different variables. I.e. Correlations between different stocks
Area Plot: Changes in several variables over time. I.e. Temperature Changes
Bubble Chart: Relationships between 3 variables using marker size. Country GDP(x), Poppulation(y) and Area(size)
'''

# Line Plot

In [ ]:
fig = px.line(df_bikes_grouped, x='monthly', y='total_bikes')
fig.show()

In [ ]:
'''
Quantity per season, weekend, holiday
Quantity over time
Humidity over time
Time variation over season
'''

In [ ]:
query = """
SELECT
    season,
    YEAR(timestamp)||LPAD(MONTH(timestamp),2, '0') AS CODMONTH,
    season || ' - ' || YEAR(timestamp)||LPAD(MONTH(timestamp),2, '0') AS season_codmonth,
    AVG(t1-t2) AS avg_temperature_variation,
    abs(AVG(t1-t2)) AS avg_abs_variation,
    AVG(wind_speed) AS avg_wind_speed,
    AVG(hum) AS avg_hum,
    SUM(cnt) total_bikes,
FROM df_bikes
GROUP BY
    season,
    YEAR(timestamp)||LPAD(MONTH(timestamp),2, '0'),
    season || ' - ' || YEAR(timestamp)||LPAD(MONTH(timestamp),2, '0')
ORDER BY
CODMONTH ASC
"""
df_bikes_transformed = conn.execute(query).df()
df_bikes_transformed.head(2)

In [ ]:
fig = px.box(df_bikes_transformed, x='season', y='total_bikes', labels={'total_bikes': 'Total Bikes'})
fig.show()

In [ ]:
fig = px.scatter(df_bikes_transformed, x='CODMONTH', y='avg_temperature_variation', size='total_bikes', labels={'total_bikes': 'Total Bikes'})
fig.show()

In [ ]:
fig = px.scatter(df_bikes_transformed, x='total_bikes', y='avg_temperature_variation', labels={'total_bikes': 'Total Bikes'})
fig.show()

In [ ]:
fig = px.histogram(df_bikes_transformed, x='total_bikes', y='avg_temperature_variation', labels={'total_bikes': 'Total Bikes'})
fig.show()

In [ ]:
fig = px.line(df_bikes_transformed, x='CODMONTH', y=['avg_hum', 'avg_wind_speed'], labels={'total_bikes': 'Total Bikes'})
fig.show()

In [ ]:
fig = px.density_heatmap(df_bikes_transformed, x='total_bikes', y='avg_hum', labels={'total_bikes': 'Total Bikes'})
fig.show()

In [ ]:
fig = px.line(df_bikes_transformed, x='CODMONTH', y='total_bikes', labels={'total_bikes': 'Total Bikes'})
fig.show()

In [ ]:
fig = px.bar(df_bikes_transformed, x='CODMONTH', y='total_bikes', labels={'total_bikes': 'Total Bikes'})
fig.show()

In [ ]:
fig = px.scatter(df_bikes_transformed, x='avg_sum', y='avg_wind_speed', labels={'avg_sum': 'Average Humidity'})
fig.show()

In [ ]:
fig = px.line(df_bikes_transformed, x='season_codmonth', y=['avg_wind_speed', 'avg_temperature_variation'])
fig.show()

In [ ]:
fig = px.line(df_bikes, x='timestamp', y='wind_speed', labels={'cnt': 'Quantity'}, title='Wind Speed over time')
fig.update_layout(width=500, height=400, template='simple_white')
# fig.update_traces(marker=dict(opacity=1.0))

In [ ]:
fig = px.bar(df_bikes, x='is_weekend', y='cnt', labels={'cnt': 'Quantity'}, color='is_weekend')
fig.update_layout(width=500, height=400, template='simple_white')
# fig.update_traces(marker=dict(opacity=1.0))

In [ ]:
fig = px.bar(df_bikes, x='season', y='cnt', labels={'cnt': 'Quantity'}, color='season')
fig.update_layout(width=500, height=400, template='simple_white')
fig.update_traces(marker=dict(opacity=1.0))

In [ ]:
fig = px.scatter(df_bikes, x='t1', y='t2', labels={'t1': 'Real Temperature', 't2': 'Perceived Temperature'})
fig.update_layout(width=500, height=300)